In [ ]:
!pip install pytorch-adapt

### Hooks Computing Features

In [ ]:
from pytorch_adapt.hooks import FeaturesHook

G = torch.nn.Linear(1000, 100)
models = {"G": G}
data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
}

hook = FeaturesHook()

losses, outputs = hook({}, {**models, **data})
# outputs contains src_imgs_features and target_imgs_features
print(outputs.keys())

losses, outputs = hook({}, {**models, **data, **outputs})
# outputs is empty
print(outputs.keys())

hook = FeaturesHook(detach=True)
losses, outputs = hook({}, {**models, **data, **outputs})
# outputs contains
# src_imgs_features_detached and target_imgs_features_detached
print(outputs.keys())

### Weighters

In [ ]:
import torch

from pytorch_adapt.weighters import MeanWeighter

weighter = MeanWeighter(weights={"y": 2.3})

logits = torch.randn(32, 512, requires_grad=True)
labels = torch.randint(0, 10, size=(32,))

x = torch.nn.functional.cross_entropy(logits, labels)
y = torch.norm(logits)

# y will by multiplied by 2.3
# x wasn't given a weight,
# so it gets multiplied by the default value of 1.
loss, components = weighter({"x": x, "y": y})
loss.backward()